In [1]:
# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import tqdm

/Users/chenyu/.pyenv/versions/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import numpy as np

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
class Linear(nn.Module):
    def __init__(self, weights,bias):
        super(Linear, self).__init__()
        self.weights = weights
        self.bias = bias        
    
    def forward(self, x):
        y = torch.matmul(x,self.weights)+self.bias
        return y

In [6]:
def min_max_normalization(df,col_name):
    min_val = df[col_name].min()
    max_val = df[col_name].max()
    df[col_name] = (df[col_name]-min_val)/(max_val-min_val)
    return df

In [7]:
def conterfactual_infer(query_x,target_y,feature_ids, model, loss_fn, max_iters):
    model.eval()
    tqdm_range = tqdm.tqdm(range(max_iters))
    cx = torch.clone(query_x)
    cx.requires_grad = True
    cf_optimizer = torch.optim.Adam([cx], lr=0.01)
    mask = torch.ones_like(query_x).type(torch.bool)
    mask[:,feature_ids] = False

    for it in tqdm_range:
        cy = model(cx)
        loss = loss_fn(cy,target_y)#loss_fn(query_x,cx,cy,target_y,feature_ids)
        loss.backward()
        cx.grad[mask]=0.
        cf_optimizer.step()
        cf_optimizer.zero_grad()

        if (it+1)%10 == 0:    
            tqdm_range.write('iter: {}, loss: {}'.format(it+1, loss.detach()))
    return cx, model(cx)

In [8]:
def query_recommendation(query_x,target_y,model,query_features,max_iters=200):
    
    if len(query_x.shape)==1:
        query_x = query_x.reshape(1,-1)
    if not isinstance(query_x,torch.Tensor):
        query_x = torch.Tensor(query_x)
        
    query_y = model(query_x)
    feature_ids = [feature_map[s] for s in query_features ]
    
    loss_fn = torch.nn.MSELoss(reduction='mean')
    
    cx,cy = conterfactual_infer(query_x,target_y,feature_ids, model, loss_fn, max_iters)
    
    diff = torch.abs(query_x-cx).reshape(-1)
    

    cf = features[diff>0]
    
    return cx,cy,cf

In [9]:
dataset = pd.read_csv("../data/trial_static_data.csv")
dataset.head()

,n_participant,duration,age>65,male,ethnicity_white,use_technology,medication,follow_up_considered,use_wearables,use_mobile_app,...,feedback_provided,prioir_treatment_history_considered,co_mobidities_considered,treatment_type,biosamples_collected,randomised,mutli_site,adverse_event_considered,support_sessions,retention_rate
0,11550.0,418.0,0.269070,0.349416,0.521052,yes,Testosterone,yes,no,no,...,yes,no,yes,behavioural,no,yes,no,yes,yes,0.532
1,60926.0,56.0,0.355991,0.461932,0.437640,no,Testosterone,yes,yes,yes,...,no,yes,yes,behavioural,yes,yes,yes,yes,yes,0.894
2,13382.0,373.0,0.404902,0.304780,0.819994,yes,5ARI,yes,yes,yes,...,yes,no,yes,behavioural,no,yes,no,no,yes,0.577
3,24097.0,183.0,0.264093,0.620426,0.824459,yes,venlafaxine,yes,no,no,...,yes,yes,yes,medication,yes,no,yes,yes,yes,0.767
4,32748.0,182.0,0.114587,0.412570,0.702339,no,citalopram,yes,yes,yes,...,no,yes,no,rTMS,no,no,no,no,yes,0.768


In [10]:
continuous = ["n_participant","duration","age>65","male","ethnicity_white","retention_rate"]
categorical = dataset.columns.difference(continuous)

In [11]:
one_hot_encoded = pd.get_dummies(dataset[categorical])
df_encoded = pd.concat([dataset.drop(categorical,axis=1), one_hot_encoded], axis=1)
df_encoded = min_max_normalization(df_encoded,["n_participant","duration"])

In [12]:
target = dataset["retention_rate"]
train_dataset, test_dataset, y_train, y_test = train_test_split(df_encoded,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                )
x_train = train_dataset.drop('retention_rate', axis=1)
x_test = test_dataset.drop('retention_rate', axis=1)


In [13]:
features = x_train.columns
feature_map = {}
for i,f in enumerate(features):
    feature_map[f]=i

In [14]:
lrg = LinearRegression()
lrg.fit(x_train,y_train)

LinearRegression()

In [15]:
pred_y = lrg.predict(x_test)
score = mean_absolute_error(y_test,pred_y)
score

3.0795921368564904e-16

In [16]:
lrg.coef_,lrg.intercept_

(array([ 5.93485115e-16, -4.96000000e-01, -1.77965045e-16, -5.17804154e-17,
         1.08995174e-16,  1.82289595e-03,  1.82289595e-03, -6.54361383e-03,
        -6.54361383e-03,  2.27529552e-04,  2.27529552e-04,  6.50009547e-03,
         6.50009547e-03,  1.60297441e-04,  1.60297441e-04,  7.32651578e-04,
         7.32651578e-04, -2.81482514e-04, -2.81482514e-04, -2.81482514e-04,
        -2.81482514e-04, -2.81482514e-04, -2.81482514e-04, -5.80242481e-03,
        -5.80242481e-03, -1.44644142e-03, -1.44644142e-03, -1.72321331e-03,
        -1.72321331e-03,  1.40330579e-03,  1.40330579e-03, -2.31720943e-02,
         2.68279057e-02,  7.39827933e-04,  7.39827933e-04,  7.39827933e-04,
         7.39827933e-04,  4.10098355e-05,  4.10098355e-05, -4.13764735e-03,
        -4.13764735e-03,  1.68131059e-04,  1.68131059e-04]),
 0.9273111728830006)

In [17]:
model = Linear(weights=torch.Tensor(lrg.coef_),bias=torch.ones(1)*lrg.intercept_)

In [18]:
### an example of query recommendation

x_test_ts = torch.Tensor(x_test.values.astype(np.float32))
y_test_ts = torch.Tensor(y_test.values.astype(np.float32))
query_x = x_test_ts[1].reshape(1,-1)
qy = model(query_x)
qy

tensor([0.6490])

In [19]:
query_features = ["n_participant","duration","age>65","male","ethnicity_white"]
target_y = 0.9

In [20]:
cx, cy, cf = query_recommendation(query_x,target_y=torch.ones(query_x.shape[0])*target_y,query_features=query_features,model=model)

100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 3618.85it/s]

iter: 10, loss: 0.042696814984083176
iter: 20, loss: 0.025372473523020744
iter: 30, loss: 0.01348830759525299
iter: 40, loss: 0.006260178983211517
iter: 50, loss: 0.0024498829152435064
iter: 60, loss: 0.0007631683256477118
iter: 70, loss: 0.0001678678090684116
iter: 80, loss: 1.7958704120246693e-05
iter: 90, loss: 1.4915716661789702e-08
iter: 100, loss: 2.8660822408710374e-06
iter: 110, loss: 3.1203217076836154e-06
iter: 120, loss: 1.549782496113039e-06
iter: 130, loss: 4.2543479139567353e-07
iter: 140, loss: 4.751274573777664e-08
iter: 150, loss: 2.25611529458547e-10
iter: 160, loss: 8.7459319786376e-09
iter: 170, loss: 7.531525625381619e-09
iter: 180, loss: 2.597122517045136e-09
iter: 190, loss: 3.218794120130042e-10
iter: 200, loss: 1.4210854715202004e-12


In [21]:
cx,cy,cf

(tensor([[0.6496, 0.0927, 0.1822, 0.5078, 0.9783, 0.0000, 1.0000, 0.0000, 1.0000,
          0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000,
          0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
          0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000]],
        requires_grad=True),
 tensor([0.9000], grad_fn=<AddBackward0>),
 Index(['duration'], dtype='object'))

In [22]:
query_x,qy

(tensor([[0.6496, 0.5988, 0.1822, 0.5078, 0.9783, 0.0000, 1.0000, 0.0000, 1.0000,
          0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000,
          0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
          0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000]]),
 tensor([0.6490]))